In [ ]:
require 'torch'
require 'gnuplot'
require 'rnn'
require 'image'
Plot = require 'itorch.Plot'

In [ ]:
gpu=1
if gpu>0 then
  print("CUDA ON")
  require 'cutorch'
  require 'cunn'
  cutorch.setDevice(gpu)
end

In [ ]:
nIters = 200
batchSize = 80
rho = 10
hiddenSize = 300
nIndex = 1
lr = 0.0001
nPredict=200

rnn = nn.Sequential()
   :add(nn.Linear(nIndex, hiddenSize))
   :add(nn.FastLSTM(hiddenSize, hiddenSize))
   :add(nn.NormStabilizer())
   :add(nn.Linear(hiddenSize, nIndex))
   :add(nn.HardTanh())
rnn = nn.Sequencer(rnn)
rnn:training()
---- print(rnn)

In [ ]:
if gpu>0 then
  rnn=rnn:cuda()
end

criterion = nn.MSECriterion()
if gpu>0 then
  criterion=criterion:cuda()
end

ii=torch.linspace(0,200, 2000)
sequence=torch.cos(ii)
if gpu>0 then
  sequence=sequence:cuda()
end

In [ ]:
offsets = {}
for i=1,batchSize do
   table.insert(offsets, math.ceil(math.random()* (sequence:size(1)-rho) ))
end
offsets = torch.LongTensor(offsets)
if gpu>0 then
  offsets=offsets:cuda()
end

--- local gradOutputsZeroed = {}
gradOutputsZeroed = {}
for step=1,rho do
  gradOutputsZeroed[step] = torch.zeros(batchSize,1)
  if gpu>0 then
    gradOutputsZeroed[step] = gradOutputsZeroed[step]:cuda()
  end
end

In [ ]:
local iteration = 5
while iteration < nIters do
   local inputs, targets = {}, {}
    inputs2, targets2 = {}, {}
   for step=1,rho do
      inputs[step] = sequence:index(1, offsets):view(batchSize,1)
      inputs2[step] = sequence:index(1, offsets):view(batchSize,1)
      offsets:add(1)
      for j=1,batchSize do
         if offsets[j] > sequence:size(1) then
            offsets[j] = 1
         end
      end
      targets2[step] = sequence:index(1, offsets)
      targets[step] = sequence:index(1, offsets)
   end
   rnn:zeroGradParameters()
   local outputs = rnn:forward(inputs)
   local err = criterion:forward(outputs[rho], targets[rho])
   ---print(string.format("Iteration %d ; NLL err = %f ", iteration, err))
   local gradOutputs = criterion:backward(outputs[rho], targets[rho])
   gradOutputsZeroed[rho] = gradOutputs
   local gradInputs = rnn:backward(inputs, gradOutputsZeroed)
   rnn:updateParameters(lr)
   iteration = iteration + 1
end

In [ ]:
rnn:evaluate()
predict=torch.FloatTensor(nPredict)
if gpu>0 then
  predict=predict:cuda()
end
for step=1,rho do
  predict[step]= sequence[step]
end

start = {}
iteration=0
while rho + iteration < nPredict do
  for step=1,rho do
    start[step] = predict:index(1,torch.LongTensor({step+iteration})):view(1,1)
  end

  output = rnn:forward(start)

  predict[iteration+rho+1] = (output[rho]:float())[1][1]

  iteration = iteration + 1
end

In [ ]:
-- scatter plots
plot = Plot():circle(torch.range(1, predict.size(predict)[1]), 
                predict, 'red', 'Loss Function'):draw()
plot:line(torch.range(1, predict.size(predict)[1]), 
                predict, 'red', ''):redraw()
plot:title('Scatter Plot Demo'):redraw()
plot:xaxis('Iterations'):yaxis('Negative Log-Likelihood'):redraw()
plot:legend(true)
plot:redraw()

# Try 2

In [ ]:
-- scatter plots
plot = Plot():circle(torch.range(1, #loss), 
                loss, 'red', 'Loss Function'):draw()
plot:line(torch.range(1, #loss), 
                loss, 'red', ''):redraw()
plot:title('Scatter Plot Demo'):redraw()
plot:xaxis('Iterations'):yaxis('Negative Log-Likelihood'):redraw()
plot:legend(true)
plot:redraw()

In [178]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

aroraname = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
wordfile = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/total_corpus_smry.csv'
nuggets = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'

m = csvigo.load({path = aroraname, mode = "large"})
w = csvigo.load({path = wordfile, mode = "large"})
q = csvigo.load({path = nuggets, mode = "large"})



N = 1000 --- Breaks at 35
K = 100
embed_dim = 3
 
dofile("utils.lua")
--- Extracting N samples
out = grabNsamples(m, N, K)
nuggs = grabNsamples(q, #q-1, nil)

mxl = 0
for k,v in pairs(out) do
    mxl = math.max(mxl, #v)
end

--- getting the length of the dictionary
vocab_size = 0
for k,v in pairs(out) do
    vocab_size = math.max(vocab_size, math.max(table.unpack(v)))
    if (k % N)==0 then
        print(k,'elements read out of ', #m)
        break
    end
end

--- Padding the data by the maximum length
xs = padZeros(out, mxl)


labels = torch.Tensor(torch.rand(#out)):reshape(#out,1)
-- labels = torch.Tensor(torch.round(torch.rand(#out))):reshape(#out,1)

--- This is the correct format to input it
input = torch.LongTensor(xs)

-- For batch inputs, it's a little easier to start with sequence-length x batch-size tensor, so we transpose songData
myDataT = input:t()
batchLSTM = nn.Sequential()
batchLSTM:add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
batchLSTM:add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
-- print(batchLSTM:forward(myDataT)) -- sanity check
-- now let's add the LSTM stuff
batchLSTM:add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
batchLSTM:add(nn.SelectTable(-1)) -- selects last state of the LSTM
batchLSTM:add(nn.Linear(embed_dim, 1)) -- map last state to a score for classification
--batchLSTM:add(nn.Sigmoid()) -- convert score to a probability
myPreds = batchLSTM:forward(myDataT)
print(#myPreds)

print(myPreds[1])
-- we can now call :backward() as follows
-- crit = nn.BCECriterion()

<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/total_corpus_smry.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	
...Utils file loaded	


1000	elements read out of 	12510	


139	elements read out of 	140	
1000	elements read out of 	12510	


 1000
    1
[torch.LongStorage of size 2]

 0.1131
[torch.DoubleTensor of size 1]



In [188]:
crit = nn.MSECriterion()
loss = 0

In [189]:
loss = loss + crit:forward(myPreds, labels)

In [190]:
grads = crit:backward(myPreds, labels)

In [ ]:
batchLSTM:backward(myDataT, grads)

In [192]:
batchLSTM:zeroGradParameters()

In [ ]:
loss = 0 
for i=1, 5, 1 do
    loss = loss + crit:forward(myPreds, labels)
    grads = crit:backward(myPreds, labels)
    batchLSTM:backward(myDataT, grads)

    print('Current loss at iteration', i,' is ', loss)
    
    --We update params at the end of each batch
    batchLSTM:updateParameters(0.05)
    batchLSTM:zeroGradParameters()
end



preds = {}
for i=1, myPreds:size()[1] do
    preds[i] = (myPreds[i][1] > 0) and 1 or 0 --- lua is stupid
end

--- Unpacking predictions and concatenating predictions into a summary
ys = unpackZeros(preds)
predsummary = buildPredSummary(ys, xs)

print(#predsummary)
print('Rouge Recall =', rougeRecall(predsummary, nuggs))
print('Rouge Precision =', rougePrecision(predsummary, nuggs))
print('Rouge F1 =', rougeF1(predsummary, nuggs))